<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам
*Этот Ноутбук является Примером/Шаблоном (Baseline) к этому соревнованию и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.


> **baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline является хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [103]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import time
import datetime
import requests as r
import json
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import pandas_profiling
import warnings
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import Series
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor  
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.base import clone

In [104]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [105]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [106]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [107]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Setup

In [108]:
VERSION    = 16
DIR_TRAIN  = '../input/cars-df/' # подключила к ноутбуку внешний датасет 
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [109]:
!ls '../input'

In [110]:
train = pd.read_csv(DIR_TRAIN + 'cars_df.csv') # датасет для обучения модели
test = pd.read_csv(DIR_TEST+'test.csv')

sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

In [111]:
train.head(5)

In [112]:
train.info()

In [113]:
test.head(5)

In [114]:
test.info()

Произвела парсинг данных с сайта https://auto.ru/moskva/cars/used/ , где размещена информация о продаже автомобилей с пробегом в Москве.

In [115]:
# START_URL='https://auto.ru/moskva/cars/used/'

# pages_url_list=[]
# while len(pages_url_list)<5000: 
#     for i in range(1,99):   
#         response = r.get( START_URL+'?page=%s'%i)
#         page = BeautifulSoup(response.text, 'html.parser')
#         for link in page.findAll('a',{"class": "Link OfferThumb"}):
#             try:
#                 if link['href'] not in pages_url_list: 
#                     pages_url_list.append(link['href'])
#             except:
#                 pass
#     print(len(pages_url_list))   
#     time.sleep(0.1)

In [116]:
# cars_list = []
# for item in pages_url_list:
#     response = r.get(item)
#     response.encoding = 'utf8'
#     page = BeautifulSoup(response.text, 'html.parser')
#     if page.find('div', class_='CardSold') == None:
#         try:
#             json_data = json.loads(
#                 page.find('script', type="application/ld+json").string)
#         except:
#             print(item)
#             pass
#         cls_str = 'CardInfoRow_'
#         span_str = 'CardInfoRow__cell'
#         try:
#             cars_list.append({
#             'bodyType': json_data['bodyType'],
#             'brand': json_data['brand'],
#             'car_url': json_data['offers']['url'],
#             'color': json_data['color'],
#             'description': json_data['description'],
#             'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
#             'enginePower': json_data['vehicleEngine']['enginePower'],
#             'fuelType': json_data['fuelType'],
#             'image': json_data['image'],
#             'mileage': page.find(
#                 'li', class_=cls_str+'kmAge').find_all('span')[1].text,
#             'modelDate': json_data['modelDate'],
#             'model_name': json_data['name'],
#             'name': json_data['vehicleEngine']['name'],
#             'numberOfDoors': json_data['numberOfDoors'],
#             'parsing_unixtime': int(time.time()),
#             'priceCurrency': json_data['offers']['priceCurrency'],
#             'productionDate': json_data['productionDate'],
#             'sell_id': page.find(
#                 'div', title='Идентификатор объявления').text,
#             'vehicleConfiguration': json_data['vehicleConfiguration'],
#             'vehicleTransmission': json_data['vehicleTransmission'],
#             'Владельцы': page.find(
#                 'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
#             'ПТС': page.find(
#                 'li', class_=cls_str+'pts').find_all('span')[1].text,
#             'Привод': page.find(
#                 'li', class_=cls_str+'drive').find_all('span')[1].text,
#             'Руль': page.find(
#                 'li', class_=cls_str+'wheel').find_all('span')[1].text,
#             'Состояние': page.find(
#                 'li', class_=cls_str+'state').find_all('span')[1].text,
#             'Таможня': page.find(
#                 'li', class_=cls_str+'customs').find_all('span')[1].text,
#             'price': page.find(
#                 'span', class_='OfferPriceCaption__price').text
#             })
#         except:
#             print(item)
#             pass
#     #else: cars_list = []

    #print('Ожидаю 0.1 секунды...')
    #time.sleep(0.1)

In [117]:
# переводим в удобный для нас вид
#df = pd.DataFrame(cars_list)
# проверяем на дубликаты, можно по двум признакам car_url или sell_id
#df['sell_id'].duplicated().sum()
# сохраняем в csv
#df.to_csv('cars_df.csv', index=False)

## Data Preprocessing

In [118]:
# первичный осмотр
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', train.shape)
display(train.head(5))
print('Размерность тестового датасета: ', test.shape)
display(test.head(5))

In [119]:
# проверим датасеты на дубликаты по уникальному идентификатору объявления
train['car_url'].duplicated().sum()
test['sell_id'].duplicated().sum()

In [120]:
#удалим столбцы с лишней информацией
test.drop(['complectation_dict', 'equipment_dict', 'model_info',
           'super_gen', 'vendor', 'Владение'], axis=1, inplace=True)

In [121]:
print('Размерность тренировочного датасета: ', train.shape)
print('Размерность тестового датасета: ', test.shape)

In [122]:
# удаляем дубликаты
train = train.drop_duplicates(subset=['car_url'])

In [123]:
# Создадим список с колонками, которые присутствуют и в train, и в test
cols_intersection = list(set(test.columns).intersection(train.columns))
cols_intersection

In [124]:
train.price.isnull().any()

In [125]:
# Переименуем test колонки
test.rename(columns={"Привод": "drivetrain",
                     "Руль": "driveSide",
                     "Состояние": "condition",
                     "Владельцы": "ownersCount",
                     "ПТС": "tcp",
                     "Таможня": "customs"
                     }, inplace=True)

# Переименуем train колонки
train.rename(columns={"Привод": "drivetrain",
                      "Руль": "driveSide",
                      "Состояние": "condition",
                      "Владельцы": "ownersCount",
                      "ПТС": "tcp",
                      "Таможня": "customs"
                      }, inplace=True)

## Label Encoding

In [126]:
test['price'] = 0
df_train = train
df_test = test

# удалим строки, где отсутствует цена
df_train = df_train.dropna(axis=0, subset=['price'])
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест

data = test.append(train, sort=False).reset_index(drop=True) # объединяем

In [127]:
data

In [128]:
data.columns

In [129]:
# еще раз проверим на дубликаты
# Для этого приведем их к одному виду int, т.к. формат в трейне и тесте отличается.
data['car_url'].value_counts()
data = data.drop_duplicates(subset=['car_url'])

In [130]:
#Будем добавлять признаки в определенный список:
#числовые:
num_cols = []

# бинарные, количество уникальных значений которых равно двум
bin_cols = []

# категориальные
cat_cols = []

## bodyType

In [131]:
data.bodyType.value_counts()

In [132]:
# приведем признак к стандартному виду, удалим лишние символы в конце строк
data['bodyType'] = data['bodyType'].astype(str).apply(lambda x: None if x.strip()=='' else x)
data['bodyType'] = data.bodyType.apply(lambda x: x.split(' ')[0].lower())

In [133]:
# проверяем
data.bodyType.value_counts()

In [134]:
#посмотрим на распределение
data.bodyType.value_counts().plot.barh() 

In [135]:
# добавим признак в категориальные
cat_cols.append('bodyType')

## brand

In [136]:
data.brand.value_counts()

In [137]:
# признак не требует предобработки,добавляем в категориальные
cat_cols.append('brand')

## color

In [138]:
data.color.value_counts()

In [139]:
#посмотрим на распределение
data.color.value_counts().plot.barh() 

In [140]:
# добавим признак в категориальные
cat_cols.append('color')

## engineDisplacement

In [141]:
data.engineDisplacement.unique()

In [142]:
#оставим только численное значение 
data.engineDisplacement = data.engineDisplacement.apply(lambda x: x[:3])
data.engineDisplacement = data.engineDisplacement.replace(' LT', '2.0')
data.engineDisplacement = data.engineDisplacement.apply(lambda x: float(x))

In [143]:
#разобьем по категориям
data["engineDisplacement"] = data["engineDisplacement"].astype('float').apply(lambda x: 4 if x >= 4 else 
   3 if x >= 3 else(2 if x >= 2 else (1 if x >= 1 else 0)))
data.engineDisplacement.value_counts()

In [144]:
# добавим признак в категориальные
cat_cols.append('engineDisplacement')

## engineDisplacement

In [145]:
# посмотрим на уникальные значения
data.enginePower.unique()

In [146]:
#преобразуем данные
data['enginePower'] = data['enginePower'].apply(lambda x: x[:3])
data['enginePower'] = data['enginePower'].replace('7 N','7')
data['enginePower'] = data['enginePower'].apply(lambda x: int(x))

In [147]:
data.enginePower.hist().barh

## fuelType

In [148]:
data.fuelType.value_counts()

In [149]:
# добавим признак в категориальные
cat_cols.append('fuelType')

## mileage

In [150]:
data.mileage.value_counts()

In [151]:
#Почистим данные
data['mileage'] = data['mileage'].apply(lambda x: str(x).replace('\xa0км', ''))
data['mileage'] = data.mileage.apply(
    lambda x: int("".join(filter(str.isdigit, x))))

In [152]:
data['mileage'].hist(figsize=(5,5), bins=50)
data.mileage.describe()

In [153]:
#Создадим категориальный признак 'mileage_cat'  
data["mileage_cat"] = data["mileage"].apply(lambda x: 4 if x>=40000 else(3 if x>=30000 else(2 if x>=20000 else (1 if x>=10000 else 0))))

In [154]:
data.mileage_cat.value_counts()

In [155]:
# добавим признаки в численные и категориальные
num_cols.append('mileage')
cat_cols.append('mileage_cat')

## productionDate / modelDate

In [156]:
data.productionDate.value_counts()

In [157]:
print(data['productionDate'].hist(figsize=(5,5), bins=50))

In [158]:
data.modelDate.value_counts()

In [159]:
print(data['modelDate'].hist(figsize=(5,5), bins=50))

In [160]:
#создадим новые признаки на основе существующих: количество лет производства и возраст модели
data['prod_age'] = 2021 - data.productionDate
data['model_age'] = 2021 - data.modelDate
#определим эти признаки в числовые данные
num_cols.append('productionDate')
num_cols.append('modelDate')
num_cols.append('prod_age')
num_cols.append('model_age')

## numberOfDoors

In [161]:
data.numberOfDoors.value_counts() #категориальный признак,не требующий доп.обработки

In [162]:
sns.countplot(x = 'numberOfDoors', data = data)

In [163]:
# добавим признак в категориальные
cat_cols.append('numberOfDoors')

## vehicleTransmission

In [164]:
data.vehicleTransmission.value_counts()

In [165]:
sns.countplot(x = 'vehicleTransmission', data = data)

In [166]:
#определяем в категориальный признак
cat_cols.append('vehicleTransmission')

## ownersCount

In [167]:
data['ownersCount'].value_counts()

In [168]:
#почистим данные, оставим только количество владельцев
data['ownersCount'] = data['ownersCount'].apply(lambda x: int(x[0])).astype('int32')

In [169]:
data.ownersCount.value_counts()

In [170]:
sns.countplot(x = 'ownersCount', data = data)

In [171]:
# добавляем признак в категориальные
cat_cols.append('ownersCount')

## tcp

In [172]:
data.tcp.value_counts()

In [173]:
# заполняем пропуски наиболее часто встречающимся значением
data.tcp = data.tcp.fillna('Оригинал')

In [174]:
# Преобразуем значения в колонке tcp
data['tcp'] = data['tcp'].apply(lambda x: 1 if x == 'Оригинал' else 0).astype('int8')
sns.countplot(x = 'tcp', data = data)

In [175]:
#определяем в бинарные признаки
bin_cols.append('tcp')

## drivetrain

In [176]:
data.drivetrain.value_counts()

In [177]:
#посмотрим на распределение
sns.countplot(x = 'drivetrain', data = data)

In [178]:
# добавляем в категориальные признаки
cat_cols.append('drivetrain')

## driveSide

In [179]:
data.driveSide.value_counts()

In [180]:
# Преобразуем значения в колонке driveSide
data['driveSide'] = data['driveSide'].apply(lambda x: 1 if x == 'Левый' else 0).astype('int8')

In [181]:
sns.countplot(x = 'driveSide', data = data)

In [182]:
# добавляем в бинарные признаки
bin_cols.append('driveSide')

## condition

In [183]:
data.condition.unique()

In [184]:
# преобразуем значения в колонке condition
data['condition'] = data['condition'].apply(lambda x: 1 if x == 'Не требует ремонта' else 0).astype('int8')
data.condition.value_counts()

In [185]:
# добавляем признак в бинарные
bin_cols.append('condition')

In [186]:
# приведем целевую переменную к числовому типу
data['price'] = data['price'].apply(lambda x: str(x))
# приведем целевую переменную к числовому типу
data.price = data.price.apply(
    lambda x: ("".join(filter(str.isdigit, x)))).astype('int')
plt.figure()
plt.title(f"Распределение {'price'}")
sns.distplot(data.price, kde=False)

## description

In [187]:
# сгенерируем числовой признак - количество символов в описании
data['description_len'] = data.description.apply(lambda x: len(str(x)))
# сам description удалим
data = data.drop('description',axis=1)
#Определим его как числовой признак
num_cols.append('description_len')

## model_name

In [188]:
# добавим признак в категориальные
cat_cols.append('model_name')

In [189]:
# посмотрим на категории
print('Категориальныe: {} \nБинарные: {}\nЧисловые: {}'.format(cat_cols, bin_cols,num_cols))

In [190]:
for colum in ['bodyType','color','model_name','brand','fuelType', 'vehicleTransmission','drivetrain', 'engineDisplacement', 'mileage_cat', 'numberOfDoors', 'ownersCount']:
    data[colum] = data[colum].astype('category').cat.codes

In [191]:
data.sample(5)

In [192]:
# Проверим значимость категориальных переменных:
temp_df = data[data['sample']==1]
imp_cat = Series(mutual_info_classif(temp_df[cat_cols], temp_df['price'],
                                     discrete_features=True), index=cat_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

In [193]:
# удалим ненужные признаки
data = data.drop(['car_url', 'image', 'name', 'parsing_unixtime', 'priceCurrency', 'sell_id', 'vehicleConfiguration', 'customs'], axis = 1)

In [194]:
data.head(5)

## Train Split

In [195]:
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample','price'], axis=1)
X = train_data.drop(['price'], axis=1)
y = train_data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_SEED)

бэггинг везде закомментирован, т.к. он не показал лучших результатов

# наивная модель

In [196]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [197]:
# Находим median по экземплярам engineDisplacement в трейне и размечаем тест
predict = X_test['engineDisplacement'].map(tmp_train.groupby('engineDisplacement')['price'].median())
VERSION = 1
#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

# CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [198]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
VERSION = 1.1
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

# 15.05%

Вот так просто со старта, даже не трогая сами данные и не подбирая настройки catboosta, получаем модель с уровнем ошибки в 18%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    
В принциепе мы можем использовать любое приобразование на целевую переменную. Например деление на курс доллара, евро или гречки :) в дату сбора данных, смотрим дату парсинга в тесте в **parsing_unixtime**

In [ ]:
np.log(y_train)

In [ ]:
model2 = CatBoostRegressor(iterations = 6000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          l2_leaf_reg = 4,
                           
                      
                         )
model2.fit(X_train, np.log(y_train),
         #cat_features=cat_features_ids,
         eval_set=(X_test, np.log(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model2.save_model('catboost_single_model_2_baseline.model')

In [ ]:
VERSION = 1.2
predict_test2 = np.exp(model2.predict(X_test))
#predict_submission6 = np.exp(model2.predict(test_data))

In [ ]:
print(f"Точность log модели по метрике MAPE: {(mape(y_test, predict_test2))*100:0.2f}%")

# 12.45%

In [ ]:
# # используем бэггинг
# from sklearn.ensemble import BaggingRegressor
# bagg_model2 = BaggingRegressor(model2, n_estimators=4, n_jobs=-1, random_state=RANDOM_SEED)
# bagg_model2.fit(X_train, np.log(y_train+1))
# predict_bagg_model2 = np.exp(bagg_model2.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_model2))*100:0.2f}%")
# VERSION = 1.3
# # predict_submission9 = np.exp(bagg_model2.predict(test_data))
# # 12.58%

# **RandomForestRegressor**

In [ ]:
# подбор параметров долго работает, закомментируем
# from sklearn.model_selection import RandomizedSearchCV
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# min_samples_split = [2, 5, 10]
# min_samples_leaf = [1, 2, 4]
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

In [ ]:
# rf = RandomForestRegressor(random_state=42)
# rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, 
#                                cv=3, verbose=2, random_state=42, n_jobs=-1)
# rf_random.fit(X_train, y_train)

In [201]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators = 1000, max_features = 'sqrt', max_depth = 30)
rf.fit(X_train, y_train)

In [ ]:
rf_pred = rf.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, rf_pred))*100:0.2f}%")

# 15.76%

In [ ]:
rf.fit(X_train, np.log(y_train))
predict_rf = np.exp(rf.predict(X_test))
print(f"Точность модели по метрике MAPE с логтаргетом: {(mape(y_test, predict_rf))*100:0.2f}%")

# 13.88%

# **Bagging for RandomForestRegression**

In [ ]:
# from sklearn.ensemble import BaggingRegressor
# bagg_rfr = BaggingRegressor(rf, n_estimators=4, n_jobs=1, random_state=RANDOM_SEED)
# bagg_rfr.fit(X_train, y_train)
# predict_bagg_rfr = bagg_rfr.predict(X_test)
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_rfr))*100:0.2f}%")

# 16.77%

In [202]:
from sklearn.ensemble import BaggingRegressor
bagg_rfr2 = BaggingRegressor(rf, n_estimators=4, n_jobs=1, random_state=RANDOM_SEED)
bagg_rfr2.fit(X_train, np.log(y_train))
predict_bagg_rfr2 = np.exp(bagg_rfr2.predict(X_test))
print(f"Точность модели по метрике MAPE с логтаргетом: {(mape(y_test, predict_bagg_rfr2))*100:0.2f}%")

# 14.47%

# **LinearRegression**

In [203]:
from sklearn.linear_model import LinearRegression
linear_regr = LinearRegression().fit(X_train, np.log(y_train+1))
lin_test = np.exp(linear_regr.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, lin_test))*100:0.2f}%")

# 25.65%

# **AdaBoostRegressor**

In [204]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(random_state=42, n_estimators=100)
ada.fit(X_train, y_train)
ada_pred = ada.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, ada_pred))*100:0.2f}%")

# 219.32%

# **GradientBoostingRegressor**

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=6, n_estimators=2000, random_state = 42, loss = 'huber', min_samples_leaf = 2, max_features = 'sqrt',
                              alpha = 0.7)
gb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(gb.predict(X_test))))*100:0.2f}%")

# 12.54%

In [ ]:
#VERSION = 1.2
#predict_submission4 = np.exp(gb.predict(test_data))

In [ ]:
# # бэггинг для градиентного бустинга
# from sklearn.ensemble import BaggingRegressor
# bagg_gb = BaggingRegressor(gb, n_estimators=4, n_jobs=-1, random_state=RANDOM_SEED)
# bagg_gb.fit(X_train, np.log(y_train+1))
# predict_bagg_gb = np.exp(bagg_gb.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_bagg_gb))*100:0.2f}%")
# VERSION = 1.3
# predict_submission5 = np.exp(bagg_gb.predict(test_data))

# **Support Vector Regression**

In [205]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
vect = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
vect.fit(X, y)
vect_pred = vect.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, vect_pred))*100:0.2f}%")

# 112.90#

# **LGBMRegressor**

In [206]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(n_estimators=1, min_child_samples=1)
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, lgbm_pred))*100:0.2f}%")

# 161.99%

In [ ]:
# хотелось бы попробовать, но долго выполняется
# regr=LazyRegressor(verbose=0,predictions=True, custom_metric=mape)
# fit and train the model
# start_time_2=time.time()
# models, predictions=regr.fit(X_train, X_test, np.log(y_train), np.log(y_test))

# **KNeighborsRegressor**

In [207]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)
neigh_pred = neigh.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, neigh_pred))*100:0.2f}%")

# 42.66%

# **xgboost**

In [ ]:
import xgboost as xgb
xb = xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03,
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)
xb.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(xb.predict(X_test))))*100:0.2f}%")
VERSION = 1.7
predict_test = np.exp(xb.predict(X_test))
predict_submission7 = np.exp(xb.predict(test_data))

# 12.12%

# Stacking

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
# выберем 3 лучшие модели
estimators = [('xb',xgb.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.5, learning_rate=0.03, 
                      max_depth=12, alpha=1, n_jobs=-1, n_estimators=1000)),
              ('gb',GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03, max_depth=6, n_estimators=2000, random_state = 42, 
                                                    loss = 'huber', min_samples_leaf = 2, max_features = 'sqrt',
                              alpha = 0.7)),
              ('model2', CatBoostRegressor(iterations = 6000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                           l2_leaf_reg = 4                
                         ))
            ]

In [ ]:
st_ensemble = StackingRegressor(estimators=estimators,final_estimator=LinearRegression())

# оцениваем точность
st_ensemble.fit(X_train, np.log(y_train))
predict_e = np.exp(st_ensemble.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_e))*100:0.2f}%")

# 12.13% - лучшее значение метрики, берем для submission

In [ ]:
VERSION = 1.8
predict_submission9 = np.exp(st_ensemble.predict(test_data))

# Submission

делаем поправку в 6%, поскольку за год подержанные автомобили подорожали в среднем на это число процентов
(согласно данным сайта https://rg.ru/2021/02/07/pochemu-poderzhannye-avtomobili-podorozhali-silnee-chem-novye.html)

In [ ]:
sample_submission['price'] = predict_submission9 * 0.94
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 27%** на ЛБ!

Большая разница в ошибке может указывать на то что тест и трейн имеют различия по выборке или то что данные в трейне могли уже устареть и их нужно обновлять.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing